<a href="https://colab.research.google.com/github/tmnguni/Coursera_Capstone/blob/main/Final_Capstone_Project_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Imported libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import folium # map rendering library
from IPython.display import HTML
import time 


In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rc('figure', dpi=300)
plt.rc('savefig', dpi=300)
fig_size = (12,6)
big_fig_size = (18,8)
fig_fc = '#ffffff'
pc = ["#4285f4", "#db4437", "#f4b400", "#0f9d58", "#ab47bc", "#00acc1", "#ff7043", 
      "#9e9d24", "#5c6bc0", "#f06292", "#00796b", "#c2185b", "#7e57c2", "#03a9f4", 
      "#8bc34a", "#fdd835", "#fb8c00", "#8d6e63", "#9e9e9e", "#607d8b"]

def plot_conf(ax, xlbl='', ylbl='', t=''):
    """
    This function perform operations to produce better-looking 
    visualizations
    """
    # changing the background color of the plot
    ax.set_facecolor('#ffffff')
    # modifying the ticks on plot axes
    ax.tick_params(axis='both', labelcolor='#616161', color='#ffffff')
    ax.tick_params(axis='both', which='major', labelsize=9)
    # adding a grid and specifying its color
    ax.grid(True, color='#e9e9e9')
    # making the grid appear behind the graph elements
    ax.set_axisbelow(True)
    # hiding axes
    ax.spines['bottom'].set_color('#ffffff')
    ax.spines['top'].set_color('#ffffff') 
    ax.spines['right'].set_color('#ffffff')
    ax.spines['left'].set_color('#ffffff')
    # setting the title, x label, and y label of the plot
    ax.set_title(t, fontsize=14, color='#616161', loc='left', pad=24, fontweight='bold');
    ax.set_xlabel(xlbl, labelpad=16, fontsize=11, color='#616161', fontstyle='italic');
    ax.set_ylabel(ylbl, color='#616161', labelpad=16, fontsize=11, fontstyle='italic');
    
# table configuration
styles = [
    dict(selector="td, th", props=[("border", "1px solid #333"), ("padding", "2px")]),
    dict(selector="th.col_heading", props=[("background", "#eee8d5"), ("color", "#b58900"), ("padding", "5px 8px")]),
    dict(selector="th.index_name", props=[("background", "#eee8d5"), ("color", "#268bd2"), ("padding", "5px 8px")]),
    dict(selector="th.blank", props=[("background", "#eee8d5"), ("color", "#268bd2"), ("padding", "0")]),
    dict(selector="th.row_heading.level0", props=[("background", "rgba(133, 153, 0, 0.1)")]),
    dict(selector="th.row_heading.level1", props=[("background", "rgba(42, 161, 152, 0.1)")]),
    dict(selector="thead tr:nth-child(2) th", props=[("border-bottom", "3px solid #333333")]),
    dict(selector="td:hover", props=[("font-weight", "bold"), ("background", "#002b36"), ("color", "Gold")]),
]

disp_fmt = "<h2 style='color: #b58900'>{}<span style='color: #FF91CE; font-size: 115%'>:</span></h2>"

# 2. Obtained, Read and Prepared data

## 2.1. Data Preparation

I first obtained the New York Data which comes in the form of json file. This was then transformed into a data frame with the following columns: Borough, Neighbourhood, Latitude and Lontitude. The following are the first five records of the Data Frame:

Obtained New York data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
NY_neighborhoods_data = newyork_data['features']
NY_neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

Transformed the New York into a dataframe and standardised columns

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

NY_neighborhoods = pd.DataFrame(columns=column_names)

for data in NY_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NY_neighborhoods = NY_neighborhoods.append({'Borough': borough,
                                                  'Neighborhood': neighborhood_name,
                                                  'Latitude': neighborhood_lat,
                                                  'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(NY_neighborhoods['Borough'].unique()),
        NY_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [7]:
NY_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# 3. **Methodology**

## 3.1. Obtained Venues

Began by first counting the neibourhoods

In [8]:
nnvc = NY_neighborhoods['Neighborhood'].value_counts()
nnvc[nnvc > 0]

Bay Terrace                  2
Murray Hill                  2
Chelsea                      2
Sunnyside                    2
Hudson Yards                 1
Greenpoint                   1
Windsor Terrace              1
Rockaway Beach               1
Lenox Hill                   1
Bedford Park                 1
Sea Gate                     1
Concourse Village            1
Elmhurst                     1
Sheepshead Bay               1
North Side                   1
Vinegar Hill                 1
Ozone Park                   1
Forest Hills                 1
Washington Heights           1
Midwood                      1
Carroll Gardens              1
Butler Manor                 1
Pleasant Plains              1
Annadale                     1
Westerleigh                  1
Broadway Junction            1
Jackson Heights              1
North Riverdale              1
Concord                      1
East Elmhurst                1
East Williamsburg            1
Jamaica Estates              1
Van Nest

Joined neibourhoods that share the same Boroughs

In [9]:
NY_neighborhoods[NY_neighborhoods['Neighborhood'] == 'Bay Terrace']

,Borough,Neighborhood,Latitude,Longitude
175,Queens,Bay Terrace,40.782843,-73.776802
235,Staten Island,Bay Terrace,40.553988,-74.139166


In [10]:
for i in range(NY_neighborhoods.shape[0]):
    nyn_ = NY_neighborhoods.loc[i, 'Neighborhood']
    if NY_neighborhoods[NY_neighborhoods['Neighborhood'] == nyn_].shape[0] > 1:
        ind_ = NY_neighborhoods[NY_neighborhoods['Neighborhood'] == nyn_].index.tolist()
        for j in ind_:
            nyb__ = NY_neighborhoods.loc[j, 'Borough']
            NY_neighborhoods.loc[j, 'Neighborhood'] = nyn_ + ', ' + nyb__

In [11]:
NY_neighborhoods[NY_neighborhoods['Neighborhood'].str.startswith('Bay Terrace')]

,Borough,Neighborhood,Latitude,Longitude
175,Queens,"Bay Terrace, Queens",40.782843,-73.776802
235,Staten Island,"Bay Terrace, Staten Island",40.553988,-74.139166


Obtained New York coordinates and map

In [12]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Using Folium.map marked New York Neighbourhoods coordinates on the map

In [13]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10, 
                         min_zoom=9, max_zoom=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(NY_neighborhoods['Latitude'], 
                                           NY_neighborhoods['Longitude'], 
                                           NY_neighborhoods['Borough'], 
                                           NY_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        weight=2,
        color='#333333',
        fill=True,
        fill_color='#ffb300',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Obtained nearby venues using Foursquare API and GetNearbyVenues() function

In [14]:
CLIENT_ID = 'EW0TI34LM2RN3W5DH54WWO410K0QSWJHLW0VGQYBFLDYP3A2' 
CLIENT_SECRET = 'EEAOAOJ2GPYQGWBBYVSRAW5WNGVNNF14FZY4V2WOVYTHNEWY' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EW0TI34LM2RN3W5DH54WWO410K0QSWJHLW0VGQYBFLDYP3A2
CLIENT_SECRET:EEAOAOJ2GPYQGWBBYVSRAW5WNGVNNF14FZY4V2WOVYTHNEWY


Used GetNearbyVenues() function to retrieve information about venues in each neighborhood using list of the names of the neighborhoods, their latitudes, as well as their longitudes and return returns a dataframe with information about each neighborhood and its venues.

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
       
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('•', end='')
            
        # create the API request URL
        url = ('https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}'
               '&v={}&ll={},{}&intent=browse&radius={}&limit={}'
               .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT))
            
        # make the GET request
        results = None
        while results is None:
            try:
                results = requests.get(url).json()["response"]["venues"]
            except:
                print('X', end='')
                results = None
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['name'], v['location']['lat'], 
                             v['location']['lng'], v['categories'][0]['name']) 
                            for v in results if len(v['categories']) > 0])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [16]:
NY_venues = getNearbyVenues(names=NY_neighborhoods['Neighborhood'],
                                   latitudes=NY_neighborhoods['Latitude'],
                                   longitudes=NY_neighborhoods['Longitude']
                                  )

••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••

removed records if venue is: Busline, building, Bus station, Bust stop or a road

In [17]:
NY_venues = NY_venues[~NY_venues['Venue Category'].isin(['Building', 'Office', 'Bus Line', 'Bus Station', 'Bus Stop', 'Road'])]
print(NY_venues.shape)

(23490, 7)


Obtained number of unique categories of venues for New York Neighbourhood

In [18]:
NY_venues.groupby('Neighborhood').size()

Neighborhood
Allerton                      82
Annadale                      77
Arden Heights                 69
Arlington                     70
Arrochar                      74
Arverne                       84
Astoria                       74
Astoria Heights               68
Auburndale                    62
Bath Beach                    79
Battery Park City             86
Bay Ridge                     83
Bay Terrace, Queens           82
Bay Terrace, Staten Island    76
Baychester                    82
Bayside                       84
Bayswater                     80
Bedford Park                  67
Bedford Stuyvesant            87
Beechhurst                    89
Bellaire                      76
Belle Harbor                  83
Bellerose                     79
Belmont                       62
Bensonhurst                   83
Bergen Beach                  80
Blissville                    76
Bloomfield                    72
Boerum Hill                   88
Borough Park                  

Obtained number of neighbourhoods vs number of unique venues

In [19]:
NY_neighborhoods.shape[0], NY_venues['Neighborhood'].unique().shape[0]

(306, 306)

examined if there are no neighbourhoods in the NY_venues dataframe that are not in the NY_neigbourhoods dataframe

In [20]:
NY_excluded_neighborhoods = set(NY_neighborhoods['Neighborhood']).difference(NY_venues['Neighborhood'])
NY_excluded_neighborhoods

set()

In [21]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 581 uniques categories.


Used one hot encoding to convert categories labels into integers

In [22]:
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

NY_onehot['Neighborhood_'] = NY_venues['Neighborhood'] 

fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighborhood_,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Service,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Baggage Locker,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campaign Office,Campground,Canal,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Football Field,College Gym,College History Building,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Stadium,College Technology Building,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Community College,Concert Hall,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Driving School,Drugstore,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,Entertainment Service,Escape Room,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hot Sp

In [23]:
NY_grouped = NY_onehot.groupby('Neighborhood_').mean().reset_index()
NY_grouped.head()

,Neighborhood_,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Service,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Baggage Locker,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campaign Office,Campground,Canal,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Football Field,College Gym,College History Building,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Stadium,College Technology Building,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Community College,Concert Hall,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Driving School,Drugstore,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,Entertainment Service,Escape Room,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hot Sp

Obtained most common venues for different neibourhoods

In [24]:
def return_most_common_venues(row, num_top_cat):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_cat]


num_top_cat = 7
indicators = ['st', 'nd', 'rd']

Created columns according to number of to venues
Created a new categories data frame

In [25]:
columns = ['Neighborhood_']
for ind in np.arange(num_top_cat):
    try:
        columns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Category'.format(ind+1))

NY_neighborhoods_categories_sorted = pd.DataFrame(columns=columns)
NY_neighborhoods_categories_sorted['Neighborhood_'] = NY_grouped['Neighborhood_']

for ind in np.arange(NY_grouped.shape[0]):
    NY_neighborhoods_categories_sorted.iloc[ind, 1:] = return_most_common_venues(
        NY_grouped.iloc[ind, :], num_top_cat)

NY_neighborhoods_categories_sorted.head()

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Allerton,Salon / Barbershop,Laundry Service,Pharmacy,Non-Profit,Car Wash,Chiropractor,Gas Station
1,Annadale,Salon / Barbershop,Pizza Place,American Restaurant,Tattoo Parlor,Nail Salon,Bar,Butcher
2,Arden Heights,Professional & Other Places,Pool,Church,Salon / Barbershop,Dentist's Office,Moving Target,General Entertainment
3,Arlington,Church,Hardware Store,Residential Building (Apartment / Condo),Professional & Other Places,Salon / Barbershop,Automotive Shop,Playground
4,Arrochar,Deli / Bodega,Food Truck,Pizza Place,Laundry Service,Beach,Restaurant,Liquor Store


Clustered the different venues using k-means and adding labels


In [26]:
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('Neighborhood_', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

kmeans.labels_[0:10]

array([4, 2, 2, 2, 2, 2, 3, 2, 0, 2], dtype=int32)

In [27]:
NY_neighborhoods_categories_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NY_merged = NY_neighborhoods.rename(columns={'Neighborhood': 'Neighborhood_'}).copy()
NY_merged = NY_merged[~NY_merged['Neighborhood_'].isin(NY_excluded_neighborhoods)]

NY_merged = NY_merged.join(NY_neighborhoods_categories_sorted.set_index('Neighborhood_'), on='Neighborhood_')

NY_merged.head()

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Bronx,Wakefield,40.894705,-73.847201,4,Salon / Barbershop,Church,Laundry Service,Doctor's Office,Food,Coworking Space,Candy Store
1,Bronx,Co-op City,40.874294,-73.829939,2,Residential Building (Apartment / Condo),School,Other Great Outdoors,Church,Salon / Barbershop,Parking,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,0,Automotive Shop,Caribbean Restaurant,Gas Station,Auto Dealership,Deli / Bodega,Diner,Church
3,Bronx,Fieldston,40.895437,-73.905643,2,College Academic Building,College Administrative Building,Residential Building (Apartment / Condo),College Quad,Cosmetics Shop,College Classroom,Tech Startup
4,Bronx,Riverdale,40.890834,-73.912585,3,Residential Building (Apartment / Condo),Synagogue,Doctor's Office,Playground,Park,Dentist's Office,Dog Run


Created a Clusterd map

In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10,
                          min_zoom=8, max_zoom=11)

rainbow = pc[:5]

for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], 
                                  NY_merged['Neighborhood_'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        weight=1,
        popup=label,
        color='#333333',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

# 3.2. Examining the clusters

In [29]:
NY_merged.loc[NY_merged['Cluster Labels'] == 0, 
               NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
2,Eastchester,Automotive Shop,Caribbean Restaurant,Gas Station,Auto Dealership,Deli / Bodega,Diner,Church
23,Longwood,Automotive Shop,Train,Pizza Place,Food,Salon / Barbershop,Gas Station,Church
24,Hunts Point,Automotive Shop,Food,Mexican Restaurant,Factory,School,Pizza Place,Gas Station
50,Gravesend,Automotive Shop,Deli / Bodega,Salon / Barbershop,Furniture / Home Store,Pizza Place,Bakery,Spa
149,College Point,Automotive Shop,Salon / Barbershop,Bank,Deli / Bodega,Doctor's Office,Mexican Restaurant,Butcher
152,Auburndale,Automotive Shop,Train,Nail Salon,Deli / Bodega,Lawyer,Athletics & Sports,Laundry Service
162,Queens Village,Food,Financial or Legal Service,Automotive Shop,Caribbean Restaurant,Salon / Barbershop,Church,Bank
164,South Jamaica,Automotive Shop,Church,Salon / Barbershop,Non-Profit,Laundry Service,Pizza Place,Grocery Store
167,Springfield Gardens,Gas Station,Automotive Shop,Church,Chinese Restaurant,Pizza Place,Deli / Bodega,High School
173,Steinway,Automotive Shop,Pizza Place,Gym / Fitness Center,Deli / Bodega,Bank,Rental Car Location,Baseball Field


In [30]:
NY_merged.loc[NY_merged['Cluster Labels'] == 1, 
               NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
8,Norwood,Doctor's Office,Park,Medical Center,Deli / Bodega,Residential Building (Apartment / Condo),Laundry Service,School
11,Pelham Parkway,Doctor's Office,Dentist's Office,Deli / Bodega,Salon / Barbershop,Pizza Place,Diner,Residential Building (Apartment / Condo)
33,Morris Park,Doctor's Office,Laundry Service,Salon / Barbershop,Residential Building (Apartment / Condo),Nail Salon,Deli / Bodega,Dentist's Office
38,Schuylerville,Doctor's Office,Salon / Barbershop,Deli / Bodega,Professional & Other Places,Pharmacy,Pizza Place,Mental Health Office
42,Pelham Gardens,Doctor's Office,Dentist's Office,Playground,Other Great Outdoors,Pizza Place,Medical Center,Pharmacy
53,Manhattan Terrace,Doctor's Office,Residential Building (Apartment / Condo),Dentist's Office,Parking,Housing Development,School,Professional & Other Places
64,Brooklyn Heights,Doctor's Office,Residential Building (Apartment / Condo),Women's Store,Salon / Barbershop,Church,School,Laundry Service
73,Starrett City,Doctor's Office,Laundry Service,Residential Building (Apartment / Condo),Parking,Pharmacy,Middle School,Gym / Fitness Center
81,Dyker Heights,Doctor's Office,Salon / Barbershop,Gas Station,Dog Run,Automotive Shop,Deli / Bodega,Event Space
98,Ocean Parkway,Residential Building (Apartment / Condo),Doctor's Office,Dentist's Office,Synagogue,Automotive Shop,Coworking Space,Deli / Bodega


In [31]:
NY_merged.loc[NY_merged['Cluster Labels'] == 2, 
               NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
1,Co-op City,Residential Building (Apartment / Condo),School,Other Great Outdoors,Church,Salon / Barbershop,Parking,Pharmacy
3,Fieldston,College Academic Building,College Administrative Building,Residential Building (Apartment / Condo),College Quad,Cosmetics Shop,College Classroom,Tech Startup
5,Kingsbridge,Laundry Service,Bank,Salon / Barbershop,Park,Other Nightlife,Thrift / Vintage Store,Financial or Legal Service
10,Baychester,Gas Station,Bank,Chinese Restaurant,Automotive Shop,Donut Shop,Deli / Bodega,Sporting Goods Shop
12,City Island,Harbor / Marina,Miscellaneous Shop,Antique Shop,Pharmacy,Deli / Bodega,Park,Residential Building (Apartment / Condo)
14,University Heights,College Academic Building,Deli / Bodega,Salon / Barbershop,Supermarket,Event Space,Emergency Room,Discount Store
16,Fordham,Clothing Store,Doctor's Office,Deli / Bodega,Salon / Barbershop,Bank,Women's Store,Medical Center
18,West Farms,Automotive Shop,Residential Building (Apartment / Condo),School,Doctor's Office,Moving Target,Sandwich Place,Chinese Restaurant
22,Port Morris,Factory,Residential Building (Apartment / Condo),Government Building,Latin American Restaurant,Event Space,Plane,Storage Facility
27,Clason Point,Park,Housing Development,Lounge,Salon / Barbershop,Automotive Shop,Intersection,Event Space


In [32]:
NY_merged.loc[NY_merged['Cluster Labels'] == 3, 
               NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
4,Riverdale,Residential Building (Apartment / Condo),Synagogue,Doctor's Office,Playground,Park,Dentist's Office,Dog Run
13,Bedford Park,Residential Building (Apartment / Condo),Deli / Bodega,Salon / Barbershop,Church,Doctor's Office,Park,Laundry Service
15,Morris Heights,Residential Building (Apartment / Condo),Grocery Store,Deli / Bodega,School,Salon / Barbershop,Spanish Restaurant,Church
20,Melrose,Residential Building (Apartment / Condo),Laundry Service,Doctor's Office,Salon / Barbershop,High School,Housing Development,Deli / Bodega
21,Mott Haven,Residential Building (Apartment / Condo),Salon / Barbershop,Grocery Store,Chinese Restaurant,Church,Asian Restaurant,Pizza Place
26,Soundview,Residential Building (Apartment / Condo),Housing Development,Laundry Service,Liquor Store,Gas Station,Pizza Place,Grocery Store
35,Spuyten Duyvil,Residential Building (Apartment / Condo),Doctor's Office,Medical Center,Dentist's Office,Salon / Barbershop,Spa,Park
54,Flatbush,Residential Building (Apartment / Condo),Salon / Barbershop,Cocktail Bar,Deli / Bodega,Dentist's Office,Bank,Church
59,Prospect Heights,Residential Building (Apartment / Condo),Thai Restaurant,Salon / Barbershop,Deli / Bodega,Mexican Restaurant,Automotive Shop,General Entertainment
65,Cobble Hill,Residential Building (Apartment / Condo),Medical Center,Playground,Taxi,Other Great Outdoors,Garden,Hospital Ward


In [33]:
NY_merged.loc[NY_merged['Cluster Labels'] == 4, 
               NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Wakefield,Salon / Barbershop,Church,Laundry Service,Doctor's Office,Food,Coworking Space,Candy Store
6,Marble Hill,Deli / Bodega,Salon / Barbershop,High School,Laundry Service,Furniture / Home Store,Church,Residential Building (Apartment / Condo)
7,Woodlawn,Bar,Deli / Bodega,Salon / Barbershop,Pub,Church,Pizza Place,Food & Drink Shop
9,Williamsbridge,Church,Salon / Barbershop,Deli / Bodega,Caribbean Restaurant,School,Nail Salon,Nightclub
17,East Tremont,Salon / Barbershop,Food,Cosmetics Shop,Shoe Store,Nail Salon,Dentist's Office,Church
19,High Bridge,Salon / Barbershop,Deli / Bodega,Pharmacy,Residential Building (Apartment / Condo),Laundry Service,Chinese Restaurant,School
25,Morrisania,Church,High School,Deli / Bodega,Residential Building (Apartment / Condo),Chinese Restaurant,Housing Development,Salon / Barbershop
30,Parkchester,Salon / Barbershop,Laundry Service,Residential Building (Apartment / Condo),Convenience Store,Indian Restaurant,School,Student Center
32,Van Nest,Deli / Bodega,Salon / Barbershop,Nail Salon,Laundry Service,Coworking Space,Residential Building (Apartment / Condo),Chinese Restaurant
34,Belmont,Salon / Barbershop,Italian Restaurant,Deli / Bodega,School,Food,Pub,Laundry Service


In [34]:
c1 = NY_merged.loc[NY_merged['Cluster Labels'] == 0,]
c1

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
2,Bronx,Eastchester,40.887556,-73.827806,0,Automotive Shop,Caribbean Restaurant,Gas Station,Auto Dealership,Deli / Bodega,Diner,Church
23,Bronx,Longwood,40.815099,-73.895788,0,Automotive Shop,Train,Pizza Place,Food,Salon / Barbershop,Gas Station,Church
24,Bronx,Hunts Point,40.809730,-73.883315,0,Automotive Shop,Food,Mexican Restaurant,Factory,School,Pizza Place,Gas Station
50,Brooklyn,Gravesend,40.595260,-73.973471,0,Automotive Shop,Deli / Bodega,Salon / Barbershop,Furniture / Home Store,Pizza Place,Bakery,Spa
149,Queens,College Point,40.784903,-73.843045,0,Automotive Shop,Salon / Barbershop,Bank,Deli / Bodega,Doctor's Office,Mexican Restaurant,Butcher
152,Queens,Auburndale,40.761730,-73.791762,0,Automotive Shop,Train,Nail Salon,Deli / Bodega,Lawyer,Athletics & Sports,Laundry Service
162,Queens,Queens Village,40.718893,-73.738715,0,Food,Financial or Legal Service,Automotive Shop,Caribbean Restaurant,Salon / Barbershop,Church,Bank
164,Queens,South Jamaica,40.696911,-73.790426,0,Automotive Shop,Church,Salon / Barbershop,Non-Profit,Laundry Service,Pizza Place,Grocery Store
167,Queens,Springfield Gardens,40.666230,-73.760421,0,Gas Station,Automotive Shop,Church,Chinese Restaurant,Pizza Place,Deli / Bodega,High School
173,Queens,Steinway,40.775923,-73.902290,0,Automotive Shop,Pizza Place,Gym / Fitness Center,Deli / Bodega,Bank,Rental Car Location,Baseball Field


In [35]:
c2 = NY_merged.loc[NY_merged['Cluster Labels'] == 1,]
c2

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
8,Bronx,Norwood,40.877224,-73.879391,1,Doctor's Office,Park,Medical Center,Deli / Bodega,Residential Building (Apartment / Condo),Laundry Service,School
11,Bronx,Pelham Parkway,40.857413,-73.854756,1,Doctor's Office,Dentist's Office,Deli / Bodega,Salon / Barbershop,Pizza Place,Diner,Residential Building (Apartment / Condo)
33,Bronx,Morris Park,40.847549,-73.850402,1,Doctor's Office,Laundry Service,Salon / Barbershop,Residential Building (Apartment / Condo),Nail Salon,Deli / Bodega,Dentist's Office
38,Bronx,Schuylerville,40.826580,-73.826203,1,Doctor's Office,Salon / Barbershop,Deli / Bodega,Professional & Other Places,Pharmacy,Pizza Place,Mental Health Office
42,Bronx,Pelham Gardens,40.862966,-73.841612,1,Doctor's Office,Dentist's Office,Playground,Other Great Outdoors,Pizza Place,Medical Center,Pharmacy
53,Brooklyn,Manhattan Terrace,40.614433,-73.957438,1,Doctor's Office,Residential Building (Apartment / Condo),Dentist's Office,Parking,Housing Development,School,Professional & Other Places
64,Brooklyn,Brooklyn Heights,40.695864,-73.993782,1,Doctor's Office,Residential Building (Apartment / Condo),Women's Store,Salon / Barbershop,Church,School,Laundry Service
73,Brooklyn,Starrett City,40.647589,-73.879370,1,Doctor's Office,Laundry Service,Residential Building (Apartment / Condo),Parking,Pharmacy,Middle School,Gym / Fitness Center
81,Brooklyn,Dyker Heights,40.619219,-74.019314,1,Doctor's Office,Salon / Barbershop,Gas Station,Dog Run,Automotive Shop,Deli / Bodega,Event Space
98,Brooklyn,Ocean Parkway,40.613060,-73.968367,1,Residential Building (Apartment / Condo),Doctor's Office,Dentist's Office,Synagogue,Automotive Shop,Coworking Space,Deli / Bodega


In [36]:
c3 = NY_merged.loc[NY_merged['Cluster Labels'] == 2,]
c3

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
1,Bronx,Co-op City,40.874294,-73.829939,2,Residential Building (Apartment / Condo),School,Other Great Outdoors,Church,Salon / Barbershop,Parking,Pharmacy
3,Bronx,Fieldston,40.895437,-73.905643,2,College Academic Building,College Administrative Building,Residential Building (Apartment / Condo),College Quad,Cosmetics Shop,College Classroom,Tech Startup
5,Bronx,Kingsbridge,40.881687,-73.902818,2,Laundry Service,Bank,Salon / Barbershop,Park,Other Nightlife,Thrift / Vintage Store,Financial or Legal Service
10,Bronx,Baychester,40.866858,-73.835798,2,Gas Station,Bank,Chinese Restaurant,Automotive Shop,Donut Shop,Deli / Bodega,Sporting Goods Shop
12,Bronx,City Island,40.847247,-73.786488,2,Harbor / Marina,Miscellaneous Shop,Antique Shop,Pharmacy,Deli / Bodega,Park,Residential Building (Apartment / Condo)
14,Bronx,University Heights,40.855727,-73.910416,2,College Academic Building,Deli / Bodega,Salon / Barbershop,Supermarket,Event Space,Emergency Room,Discount Store
16,Bronx,Fordham,40.860997,-73.896427,2,Clothing Store,Doctor's Office,Deli / Bodega,Salon / Barbershop,Bank,Women's Store,Medical Center
18,Bronx,West Farms,40.839475,-73.877745,2,Automotive Shop,Residential Building (Apartment / Condo),School,Doctor's Office,Moving Target,Sandwich Place,Chinese Restaurant
22,Bronx,Port Morris,40.801664,-73.913221,2,Factory,Residential Building (Apartment / Condo),Government Building,Latin American Restaurant,Event Space,Plane,Storage Facility
27,Bronx,Clason Point,40.806551,-73.854144,2,Park,Housing Development,Lounge,Salon / Barbershop,Automotive Shop,Intersection,Event Space


In [37]:
c4 = NY_merged.loc[NY_merged['Cluster Labels'] == 3,]
c4

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
4,Bronx,Riverdale,40.890834,-73.912585,3,Residential Building (Apartment / Condo),Synagogue,Doctor's Office,Playground,Park,Dentist's Office,Dog Run
13,Bronx,Bedford Park,40.870185,-73.885512,3,Residential Building (Apartment / Condo),Deli / Bodega,Salon / Barbershop,Church,Doctor's Office,Park,Laundry Service
15,Bronx,Morris Heights,40.847898,-73.919672,3,Residential Building (Apartment / Condo),Grocery Store,Deli / Bodega,School,Salon / Barbershop,Spanish Restaurant,Church
20,Bronx,Melrose,40.819754,-73.909422,3,Residential Building (Apartment / Condo),Laundry Service,Doctor's Office,Salon / Barbershop,High School,Housing Development,Deli / Bodega
21,Bronx,Mott Haven,40.806239,-73.916100,3,Residential Building (Apartment / Condo),Salon / Barbershop,Grocery Store,Chinese Restaurant,Church,Asian Restaurant,Pizza Place
26,Bronx,Soundview,40.821012,-73.865746,3,Residential Building (Apartment / Condo),Housing Development,Laundry Service,Liquor Store,Gas Station,Pizza Place,Grocery Store
35,Bronx,Spuyten Duyvil,40.881395,-73.917190,3,Residential Building (Apartment / Condo),Doctor's Office,Medical Center,Dentist's Office,Salon / Barbershop,Spa,Park
54,Brooklyn,Flatbush,40.636326,-73.958401,3,Residential Building (Apartment / Condo),Salon / Barbershop,Cocktail Bar,Deli / Bodega,Dentist's Office,Bank,Church
59,Brooklyn,Prospect Heights,40.676822,-73.964859,3,Residential Building (Apartment / Condo),Thai Restaurant,Salon / Barbershop,Deli / Bodega,Mexican Restaurant,Automotive Shop,General Entertainment
65,Brooklyn,Cobble Hill,40.687920,-73.998561,3,Residential Building (Apartment / Condo),Medical Center,Playground,Taxi,Other Great Outdoors,Garden,Hospital Ward


In [38]:
c5 = NY_merged.loc[NY_merged['Cluster Labels'] == 4,]
c5

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Bronx,Wakefield,40.894705,-73.847201,4,Salon / Barbershop,Church,Laundry Service,Doctor's Office,Food,Coworking Space,Candy Store
6,Manhattan,Marble Hill,40.876551,-73.910660,4,Deli / Bodega,Salon / Barbershop,High School,Laundry Service,Furniture / Home Store,Church,Residential Building (Apartment / Condo)
7,Bronx,Woodlawn,40.898273,-73.867315,4,Bar,Deli / Bodega,Salon / Barbershop,Pub,Church,Pizza Place,Food & Drink Shop
9,Bronx,Williamsbridge,40.881039,-73.857446,4,Church,Salon / Barbershop,Deli / Bodega,Caribbean Restaurant,School,Nail Salon,Nightclub
17,Bronx,East Tremont,40.842696,-73.887356,4,Salon / Barbershop,Food,Cosmetics Shop,Shoe Store,Nail Salon,Dentist's Office,Church
19,Bronx,High Bridge,40.836623,-73.926102,4,Salon / Barbershop,Deli / Bodega,Pharmacy,Residential Building (Apartment / Condo),Laundry Service,Chinese Restaurant,School
25,Bronx,Morrisania,40.823592,-73.901506,4,Church,High School,Deli / Bodega,Residential Building (Apartment / Condo),Chinese Restaurant,Housing Development,Salon / Barbershop
30,Bronx,Parkchester,40.837938,-73.856003,4,Salon / Barbershop,Laundry Service,Residential Building (Apartment / Condo),Convenience Store,Indian Restaurant,School,Student Center
32,Bronx,Van Nest,40.843608,-73.866299,4,Deli / Bodega,Salon / Barbershop,Nail Salon,Laundry Service,Coworking Space,Residential Building (Apartment / Condo),Chinese Restaurant
34,Bronx,Belmont,40.857277,-73.888452,4,Salon / Barbershop,Italian Restaurant,Deli / Bodega,School,Food,Pub,Laundry Service


In [39]:
c1.groupby(['1st Most Common Category']).size().reset_index(name='Counts')

,1st Most Common Category,Counts
0,Automotive Shop,15
1,Deli / Bodega,1
2,Food,1
3,Gas Station,3


In [40]:
c2.groupby(['1st Most Common Category',]).size().reset_index(name = 'Count')

,1st Most Common Category,Count
0,College Residence Hall,1
1,Deli / Bodega,1
2,Dentist's Office,1
3,Doctor's Office,32
4,Eye Doctor,1
5,Hospital,1
6,Residential Building (Apartment / Condo),2


In [41]:
c3.groupby(['1st Most Common Category',]).size().reset_index(name = 'Count')

,1st Most Common Category,Count
0,Art Gallery,3
1,Automotive Shop,1
2,Bank,3
3,Bar,2
4,Baseball Field,1
5,Beach,7
6,Boat or Ferry,3
7,Boutique,1
8,Chinese Restaurant,3
9,Church,3


In [42]:
c4.groupby(['1st Most Common Category',]).size().reset_index(name = 'Count')

,1st Most Common Category,Count
0,Government Building,1
1,Residential Building (Apartment / Condo),40


In [43]:
c5.groupby(['1st Most Common Category',]).size().reset_index(name = 'Count')

,1st Most Common Category,Count
0,Automotive Shop,1
1,Bar,1
2,Church,2
3,Deli / Bodega,9
4,Doctor's Office,1
5,Residential Building (Apartment / Condo),2
6,Salon / Barbershop,39
